In [2]:
import pandas as pd
from time import mktime, strptime
import datetime

In [3]:
DATABASE_ADDRESS = "../Data/Raw/chartevents.csv"
PROCEDURE_EVENTS_ADDRESS = "../Data/Raw/procedureevents.csv"
CODE_MAPPING_ADDRESS = "../Data/Raw/code_mappings.csv"
DATA_SIZE = 2*1000000

###### 20大概需要14分鐘
###### 1需要30.5MB
###### 大檔案有 329,499789行

In [4]:
FILENAME = datetime.datetime.now().strftime('%Y%m%d') + f'_{DATA_SIZE:.0e}'
FILENAME

'20230709_2e+06'

In [5]:
item = dict()
mark = tuple()

In [6]:
df = pd.read_csv(CODE_MAPPING_ADDRESS, engine="python", encoding="unicode_escape")
id_to_label = dict(zip(df["itemid"], df["label"]))
label_to_id = dict(zip(df["label"], df["itemid"]))
# df.head()

In [7]:
id_to_label

{220003: 'ICU Admission date',
 220045: 'Heart Rate',
 220046: 'Heart rate Alarm - High',
 220047: 'Heart Rate Alarm - Low',
 220048: 'Heart Rhythm',
 220050: 'Arterial Blood Pressure systolic',
 220051: 'Arterial Blood Pressure diastolic',
 220052: 'Arterial Blood Pressure mean',
 220056: 'Arterial Blood Pressure Alarm - Low',
 220058: 'Arterial Blood Pressure Alarm - High',
 220059: 'Pulmonary Artery Pressure systolic',
 220060: 'Pulmonary Artery Pressure diastolic',
 220061: 'Pulmonary Artery Pressure mean',
 220063: 'Pulmonary Artery Pressure Alarm - High',
 220066: 'Pulmonary Artery Pressure Alarm - Low',
 220069: 'Left Artrial Pressure',
 220072: 'Central Venous Pressure Alarm - High',
 220073: 'Central Venous Pressure  Alarm - Low',
 220074: 'Central Venous Pressure',
 220088: 'Cardiac Output (thermodilution)',
 220125: 'Left Ventricular Assit Device Flow',
 220128: 'Right Ventricular Assist Device Flow',
 220179: 'Non Invasive Blood Pressure systolic',
 220180: 'Non Invasive Bl

In [8]:
mark = ("Heart Rate", \
        "Respiratory Rate", \
        "Non Invasive Blood Pressure systolic", \
        "Non Invasive Blood Pressure diastolic", \
        "O2 saturation pulseoxymetry", \
        "Temperature Celsius", \
        "Temperature Fahrenheit")

In [9]:
%%time 
df = pd.read_csv(DATABASE_ADDRESS, usecols=["hadm_id", "charttime", "itemid", "value"], engine="python", encoding="unicode_escape", nrows=DATA_SIZE)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   hadm_id    int64  
 1   charttime  object 
 2   itemid     int64  
 3   value      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 61.0+ MB
CPU times: user 6.44 s, sys: 1.17 s, total: 7.6 s
Wall time: 8.13 s


In [10]:
df['charttime'] = pd.to_datetime(df['charttime']).round("10min")

In [11]:
df = df[df["itemid"].isin((220045, 220210, 220179, 220180, 220277, 223762, 223761))]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 604291 entries, 12 to 1999999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   hadm_id    604291 non-null  int64         
 1   charttime  604291 non-null  datetime64[ns]
 2   itemid     604291 non-null  int64         
 3   value      604291 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 23.1 MB


In [13]:
# 建立一個CLASS容器來裝每一個PATIENT
class Patient:
    data = dict()
    def __init__(self, ):
        self.data = {"hadm_id":None, "charttime":None}
        for label_ in mark:
            self.data[label_] = None
    def allna(self, ):
        return all ([self.data[i]==None for i in mark])

In [14]:
output = []
cnt = 0
tp = [label_to_id[i] for i in mark]
for name, group in df.groupby(["hadm_id", "charttime"]):
    patient = Patient()
    
    for id_, value_ in zip(group.itemid, group.value):
        cnt += 1
        patient.data[id_to_label[id_]] = value_
    patient.data["hadm_id"], patient.data["charttime"] = name
    if patient.data["Temperature Fahrenheit"] != None:
        patient.data["Temperature Celsius"] = round((float(patient.data["Temperature Fahrenheit"]) - 32) * 5 / 9, 1)
    output.append(patient.data)
merged_df = pd.DataFrame(output,columns=['hadm_id','charttime']+list(mark)).drop(["Temperature Fahrenheit"], axis=1)
merged_df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_pivot.csv", float_format='%.2f', index=0)

In [15]:
# df.to_csv("chart_test3_merge_new.csv", float_format='%.2f', index=0)
print("amount: ", len(output))
print("amount: ", len(merged_df))
# merged_df

amount:  140901
amount:  140901


In [16]:
df.sample(5)

,hadm_id,charttime,itemid,value
1625647,28394491,2146-05-09 01:00:00,220277,96.0
582360,29292100,2164-02-08 18:20:00,220179,122.0
1556795,25883588,2113-02-08 07:00:00,220045,102.0
1530011,23547619,2157-09-30 04:00:00,220179,83.0
1748340,29437169,2169-10-11 19:00:00,220179,109.0


In [17]:
df = pd.read_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_pivot.csv", engine="python", encoding="unicode_escape", parse_dates=["charttime"])
# df.dropna(thresh=5, inplace=True)
for name, group in df.groupby("hadm_id"):
    group.interpolate(method='bfill', inplace=True)
    group.interpolate(method='ffill', inplace=True)
    df.update(group)
df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_filled.csv", float_format='%.1f', index=0)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140901 entries, 0 to 140900
Data columns (total 8 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   hadm_id                                140901 non-null  int64         
 1   charttime                              140901 non-null  datetime64[ns]
 2   Heart Rate                             140901 non-null  float64       
 3   Respiratory Rate                       140901 non-null  float64       
 4   Non Invasive Blood Pressure systolic   140692 non-null  float64       
 5   Non Invasive Blood Pressure diastolic  140692 non-null  float64       
 6   O2 saturation pulseoxymetry            140897 non-null  float64       
 7   Temperature Celsius                    140604 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 8.6 MB


In [19]:
def read_csv_function(road: str) -> pd.core.frame.DataFrame:
    print("Reading", road, "...")
    return pd.read_csv(road, engine='python')
def to_timestamp(s: str):
    try:
        return int(mktime(strptime(s, "%Y-%m-%d %H:%M:%S")))
    except ValueError:
        try:
            return int(mktime(strptime(s, "%Y-%m-%d %H:%M")))
        except ValueError:
            return int(mktime(strptime(s, "%Y/%m/%d %H:%M")))

In [25]:
df = read_csv_function("../Data/Raw/CPR pt.csv")
dic = {}
for i, it in enumerate(df.iterrows()):
    if df.iloc[i].hadm_id not in dic:
        dic[df.iloc[i].hadm_id] = list()
    dic[df.iloc[i].hadm_id].append(to_timestamp(str(df.iloc[i].chartdate)))

df = read_csv_function(f"../Data/Preprocessed/chartevents_{FILENAME}_filled.csv")
ans = [False]*len(df)
for i in range(len(df)):
    if int(df.iloc[i].hadm_id) in dic:
        for j in dic[df.iloc[i].hadm_id]:
            if 0 <= j - to_timestamp(str(df.iloc[i].charttime)) <= 14400:
                ans[i] = True
                break

df["label"] = ans
df.to_csv(f"../Data/Preprocessed/chartevents_{FILENAME}_labeled.csv", float_format='%.2f', index=0)

Reading ../Data/Raw/CPR pt.csv ...


ValueError: time data '6/13/2138' does not match format '%Y/%m/%d %H:%M'

In [ ]:
df.label.value_counts()

label
False    70473
True        25
Name: count, dtype: int64